# Spark with Scala
Apache Toree - Jupyter Notebook testing

In [1]:
sc.parallelize(1 to 100).map(x => x*x).take(10)

Array(1, 4, 9, 16, 25, 36, 49, 64, 81, 100)

## Word Counter

In [3]:
val lines = Array("hello world", "hello one", "hello two")
sc.parallelize(lines).
  flatMap(line => line.split("\\W+")).
  map(word => (word,1)).
  reduceByKey(_+_).
  sortBy({case (word,count) => count},false).
  collect.
  mkString("\n")

(hello,3)
(two,1)
(one,1)
(world,1)